In [ ]:
!pip install torchvision

In [1]:

import numpy as np
import autokeras as ak
from keras.preprocessing.image import ImageDataGenerator
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_tensor
from torchvision.transforms import v2

In [4]:
data_dir = 'Dog Breed Dataset'

# Use ImageDataGenerator to load and preprocess the images
datagen = ImageDataGenerator(rescale = 1. / 255)

# Define the data generators for training, validation, and testing
train_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/train',
    target_size = (256, 256),
    batch_size = 16,
    class_mode = 'categorical'
)


test_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/test',
    target_size = (256, 256),
    batch_size = 1,
    class_mode = 'categorical',
    shuffle = False
)

Found 322 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


In [5]:


# Initialize empty lists for training and test data and labels
x_train, y_train = [], []
x_test, y_test = [], []

# Load training data in batches
for _ in range(len(train_generator)):
    batch_x, batch_y = train_generator.next()
    x_train.append(batch_x)
    y_train.append(batch_y)

# Load test data in batches
for _ in range(len(test_generator)):
    batch_x, batch_y = test_generator.next()
    x_test.append(batch_x)
    y_test.append(batch_y)



In [6]:
# Concatenate the training and test data
x_train = np.concatenate(x_train)
x_test = np.concatenate(x_test)

# Concatenate the labels and then convert to integer labels
y_train = np.concatenate(y_train)
y_train = np.argmax(y_train, axis=1)

y_test = np.concatenate(y_test)
y_test = np.argmax(y_test, axis=1)


In [7]:
clf = ak.ImageClassifier(
    max_trials = 5,
    objective = 'val_accuracy',
    overwrite = True,
)

# Search for the best model architecture
clf.fit(x_train, y_train, epochs = 10)

Trial 1 Complete [00h 09m 55s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 09m 55s


Epoch 1/10
11/11 [==============================] - 51s 4s/step - loss: 4.7765 - accuracy: 0.8106
Epoch 2/10
11/11 [==============================] - 49s 4s/step - loss: 6.3426e-04 - accuracy: 1.0000
Epoch 3/10
11/11 [==============================] - 79s 7s/step - loss: 2.5901e-06 - accuracy: 1.0000
Epoch 4/10
11/11 [==============================] - 65s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
11/11 [==============================] - 62s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
11/11 [==============================] - 62s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
11/11 [==============================] - 49s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
11/11 [==============================] - 49s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
11/11 [==============================] - 53s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
11/11 [==============================] - 56s 5s/step - loss: 0.0000e+00 - 

INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


In [8]:
results = clf.evaluate(x_test, y_test)
print(results)
print("Test accuracy:", results[1])

2/2 [==============================] - 2s 631ms/step - loss: 1.1038e-08 - accuracy: 1.0000
[1.1037893266063747e-08, 1.0]
Test accuracy: 1.0


In [9]:
from sklearn.metrics import classification_report

In [10]:
y_pred = clf.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, accuracy, and F1 score
report = classification_report(y_test, y_pred_classes)
print(report)

2/2 [==============================] - 2s 867ms/step
              precision    recall  f1-score   support

           0       0.33      1.00      0.50        18
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        18

    accuracy                           0.33        54
   macro avg       0.11      0.33      0.17        54
weighted avg       0.11      0.33      0.17        54



c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:

clf.export_model().save("Dog Breed Model")